In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-surprise
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
nltk.download("stopwords")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from PIL import Image
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
_file='/content/drive/MyDrive/Colab Notebooks/CSE488/Preprocessed_data.csv'

In [ ]:
book_rec=pd.read_csv(_file)
book_rec.head(9)

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada
5,5,67544,"toronto, ontario, canada",30.0000,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],toronto,ontario,canada
6,6,85526,"victoria, british columbia, canada",36.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],victoria,british columbia,canada
7,7,96054,"ottawa, ontario, canada",29.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
8,8,116866,"ottawa, ,",34.7439,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,",",NaN


In [ ]:
book_rec.isnull().sum()

Unnamed: 0                 0
user_id                    0
location                   0
age                        0
isbn                       0
rating                     0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_s                      0
img_m                      0
img_l                      0
Summary                    0
Language                   0
Category                   0
city                   14103
state                  22798
country                35374
dtype: int64

In [ ]:
book_rec.dropna(inplace=True)
book_rec.reset_index(drop=True, inplace=True)

In [ ]:
book_rec.drop(columns = ['Unnamed: 0','location',
                   'img_s','img_m','img_l','age',
                   'state','Language','country',
                       ],axis=1,inplace = True)


book_rec.drop(index=book_rec[book_rec['rating'] == 0].index, inplace=True)

book_rec['Category'] = book_rec['Category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())

book_rec.head(5)

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Category,city
1,8,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",Actresses,timmins
4,67544,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",Actresses,toronto
7,123629,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",Actresses,kingston
9,200273,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",Actresses,comber
10,210926,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",Actresses,guelph


In [ ]:

rating_count = book_rec.groupby('isbn')['rating'].count().reset_index()
rating_count.rename(columns={'rating': 'Total-Rating-Count'}, inplace=True)

average_ratings = book_rec.groupby('isbn')['rating'].mean().reset_index()
average_ratings.rename(columns={'rating': 'Average-Rating'}, inplace=True)
rating_popularity = pd.merge(rating_count, average_ratings, on='isbn', how='left')
merged_dataset = pd.merge(rating_popularity, book_rec[['isbn', 'book_title', 'book_author']], on='isbn', how='left').drop_duplicates()

threshold = 150
popular_books = merged_dataset[merged_dataset['Total-Rating-Count'] >= threshold]

sorted_books = popular_books.sort_values(by=['Total-Rating-Count', 'Average-Rating'], ascending=False)

try:
    num_books_to_show = int(input("Enter the number of books you want to see: "))
except ValueError:
    print("Invalid input")
    num_books_to_show = 5

top_books = sorted_books.head(num_books_to_show)
top_books[['isbn', 'book_title', 'book_author', 'Total-Rating-Count', 'Average-Rating']]


Enter the number of books you want to see: 5


,isbn,book_title,book_author,Total-Rating-Count,Average-Rating
55900,0316666343,The Lovely Bones: A Novel,Alice Sebold,657,8.191781
309276,0971880107,Wild Animus,Rich Shapero,548,4.405109
109057,0385504209,The Da Vinci Code,Dan Brown,470,8.434043
43923,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,363,8.192837
9902,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,312,7.875000


In [ ]:

def probable_rating(user_id, isbn):
    user_book_data = book_rec[(book_rec['user_id'] == user_id) & (book_rec['isbn'] == isbn)]
    probable_rating = user_book_data['rating'].mean()

    if not np.isnan(probable_rating):
        return f"Probable rating of the book ({isbn}) by user ({user_id}): {probable_rating:.2f}"
    else:
        return "No rating found for the specified user and book combination"
user_id_input = input("Enter User-ID: ")
isbn_input = input("Enter ISBN: ")

result = probable_rating(int(user_id_input), isbn_input)
print(result)


Enter User-ID: 8
Enter ISBN: ASDCZBF
No rating found for the specified user and book combination


In [ ]:
from surprise import Reader, Dataset, KNNBasic
sample_fraction = 0.05
file_path = '/content/drive/MyDrive/Colab Notebooks/CSE488/Preprocessed_data.csv'
df = pd.read_csv(file_path).sample(frac=sample_fraction, random_state=42)
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['user_id', 'isbn', 'rating']], reader)

sim_options = {
    'name': 'pearson',
    'user_based': True
}

algo = KNNBasic(sim_options=sim_options)
algo.fit(data.build_full_trainset())

def get_recommendations(user_id, k=10):
    if user_id not in algo.trainset._raw2inner_id_users:
        print(f"User {user_id} is not in the training set.")
        return
    neighbors = algo.get_neighbors(algo.trainset.to_inner_uid(user_id), k=k)

    user_books = df[df['user_id'] == user_id]['isbn'].unique()
    predictions = []
    for neighbor_id in neighbors:
        neighbor_books = df[df['user_id'] == algo.trainset.to_raw_uid(neighbor_id)]['isbn'].unique()
        common_books = set(user_books) & set(neighbor_books)

        if common_books:
            avg_rating = df[(df['user_id'] == algo.trainset.to_raw_uid(neighbor_id)) & (df['isbn'].isin(common_books))]['rating'].mean()
            predictions.append((common_books, avg_rating))

    predictions.sort(key=lambda x: x[1], reverse=True)

    top_recommendations = predictions[:k]

    if top_recommendations:
        print(f"Top {k} recommendations for user {user_id}:")
        for books, rating in top_recommendations:
            print(f"Common Books: {books}, Average Rating: {rating}")
    else:
        print(f"No recommendations found for user {user_id}.")

user_id = 11676
get_recommendations(user_id)


Computing the pearson similarity matrix...
Done computing similarity matrix.
Top 10 recommendations for user 11676:
Common Books: {'0671525743', '0553582127'}, Average Rating: 8.0
Common Books: {'0679432477', '0140075577'}, Average Rating: 7.5
Common Books: {'0304345350', '0553295608'}, Average Rating: 5.0
Common Books: {'0553582755', '0517540177'}, Average Rating: 4.0
Common Books: {'0515128554', '0515130923'}, Average Rating: 4.0
Common Books: {'0671525743', '0345413873'}, Average Rating: 2.5
